In [5]:
!pip install pymongo
!pip install neo4j



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 2.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from pymongo import MongoClient
import os
import pandas as pd
from neo4j import GraphDatabase

In [6]:
from app.database.neo4j_client import driver

In [19]:
with driver.session() as sess:
    res = sess.run("""
MATCH (p:Producto)
WHERE NOT (p)<-[:REALIZA_PEDIDO]-(:Orden)
RETURN p.codigo, p.nombre, p.marca, p.precio
    """)
    proveedores_json = [dict(record) for record in res]

proveedores_json

[{'p.codigo': 90,
  'p.nombre': 'Guiso',
  'p.marca': 'Pizzini',
  'p.precio': 193.02}]

In [ ]:
with driver.session() as sess:
    res = sess.run("""
        MATCH (p:Proveedor)
        WHERE NOT (p)<-[:LE_COMPRA_A]-(:Orden)
        RETURN p.id_proveedor AS id_proveedor, p.razon_social AS razon_social, p.activo AS activo, p.habilitado AS habilitado
    """)
    proveedores_json = [dict(record) for record in res]

proveedores_json

In [ ]:
with driver.session() as sess:
    res = sess.run("""
        MATCH (p:Proveedor)
        WHERE NOT (p)<-[:LE_COMPRA_A]-(:Orden)
        RETURN p.id_proveedor AS id_proveedor, p.razon_social AS razon_social, p.activo AS activo, p.habilitado AS habilitado
    """)
    proveedores_json = [dict(record) for record in res]

proveedores_json

In [ ]:
with driver.session() as sess:
    res = sess.run("""
        MATCH (p:Proveedor)
        WHERE NOT (p)<-[:LE_COMPRA_A]-(:Orden)
        RETURN p.id_proveedor AS id_proveedor, p.razon_social AS razon_social, p.activo AS activo, p.habilitado AS habilitado
    """)
    proveedores_json = [dict(record) for record in res]

proveedores_json

In [8]:
mongo = MongoClient(os.getenv("MONGO_URI", "mongodb://localhost:27017"))

In [9]:
mdb = mongo["backoffice"]

In [10]:
df_prov = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
df_tel  = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
df_prod = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
df_op   = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
df_det  = pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")

In [11]:
res = pd.merge(df_prov,df_tel, on="id_proveedor",how="inner")
proveedores = res.groupby("id_proveedor").apply(lambda g: {
    'id_proveedor': int(g['id_proveedor'].iloc[0]),
    "CUIT_proveedor": int(g["CUIT_proveedor"].iloc[0]),
    "razon_social":g["razon_social"].iloc[0],
    "tipo_sociedad":g["tipo_sociedad"].iloc[0] ,
    "direccion":g["direccion"].iloc[0] , 
    "activo" :int(g["activo"].iloc[0]),
    "habilitado":int(g["habilitado"].iloc[0]),
    "telefonos": g[['codigo_area', 'nro_telefono', 'tipo']].to_dict(orient='records')
}).reset_index(name='document')["document"].to_list()

In [12]:
res = pd.merge(df_op,df_det,on="id_pedido",how="inner")
ordenes = res.groupby('id_pedido').apply(lambda g: {
    "id_pedido": int(g['id_pedido'].iloc[0]),
    'id_proveedor': int(g['id_proveedor'].iloc[0]),
    'fecha': g['fecha'].iloc[0],
    'iva': float(g['iva'].iloc[0]),
    'items': g[['id_producto', 'nro_item', 'cantidad']].to_dict(orient='records')
}).reset_index(name='document')["document"].to_list()

In [13]:

def import_mongo():

    mdb.proveedores.delete_many({})
    mdb.proveedores.insert_many(proveedores)

    mdb.productos.delete_many({})
    mdb.productos.insert_many(df_prod.to_dict("records"))
    

    mdb.ordenes.delete_many({})
    mdb.ordenes.insert_many(ordenes)

    
import_mongo()

In [14]:
uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
neo = GraphDatabase.driver(uri, auth=("neo4j", os.getenv("NEO4J_PASSWORD", "epicpassword123")))

In [15]:
def import_neo4j():
    with neo.session() as sess:
        sess.run("MATCH (n) DETACH DELETE n")
        # proveedores
        df = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (p:Proveedor {
                  id_proveedor:$id_proveedor,
                  cuit: $CUIT_proveedor,
                  razon_social: $razon_social,
                  activo: $activo,
                  habilitado: $habilitado
                })
            """, **r.to_dict())

        # RELACIÓN TELEFONO
        df = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                CREATE (p)-[:TIENE_TELEFONO {numero:$telefono}]->(:Telefono)
            """, id_proveedor=r.id_proveedor, telefono=r.nro_telefono)

        # productos
        df = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (:Producto {
                  codigo: $id_producto,
                  nombre: $descripcion,
                  marca: $marca,
                  precio: $precio
                })
            """, **r.to_dict())

        # ordenes y detalles
        df_op = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
        df_op["fecha"] = pd.to_datetime(df_op["fecha"], dayfirst=True).dt.strftime("%Y-%m-%d")
        df_det= pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")
        
        for _,o in df_op.iterrows():
            sess.run("""
                CREATE (o:Orden {
                     id_pedido:$id_pedido,
                     id_proveedor:$id_proveedor,
                     iva:$iva,
                  fecha: date($fecha)
                })
                WITH o
                MATCH (p:Proveedor {id_proveedor:$id_proveedor})
                CREATE (o) - [:LE_COMPRA_A] -> (p)
            """, **o.to_dict())

        #Orden tiene (3 de este) producto
        
        for _,detalle in df_det.iterrows():
            sess.run("""
                MATCH (o:Orden {id_pedido:$id_pedido})
                MATCH (p:Producto {codigo: $id_producto})
                CREATE (o)-[:REALIZA_PEDIDO {cantidad: $cantidad}] -> (p)
            """, **detalle.to_dict())


        for _,d in df_det.iterrows():
            sess.run("""
                MATCH (o:Orden {id_pedido:$id_pedido,id_producto:$id_producto}), (pr:Producto {codigo:$id_producto})
                CREATE (o)-[:CONTiene {cantidad:$cantidad}]->(pr)
            """, **d.to_dict())
import_neo4j()